### s01 set up 

In [0]:
import pyspark 
import os
from pyspark import sql
from pyspark.sql import functions as f
from pyspark.sql import Window

In [0]:
# ------------ login to snowflake
password = dbutils.secrets.get(scope = "auea-kv-sbx-dxdtlprdct01", key = "sfdbrsdskey")

options = {
  "sfUrl": "vodafonenz_prod.australia-east.azure.snowflakecomputing.com/", 
  "sfUser": "SVC_LAB_DS_DATABRICKS",
  "pem_private_key": password.replace('\\n', '\n'),
  "sfDatabase": "PROD_ACCOUNT_RISK",
  "sfSchema": "SERVING",
  "sfWarehouse": "PROD_ACCOUNT_RISK_WH"
}


### s02 directory 

In [0]:
dir_audience_conrtol_01 = '/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_audience_control_20241114'
dir_audience_conrtol_02 = '/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_audience_control_20241128'
dir_audience_conrtol_03 = '/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_audience_control_20241205'

dir_audience_treatment_01 = '/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_audience_treatment_20241114'
dir_audience_treatment_02 = '/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_audience_treatment_20241128'
dir_audience_treatment_03 = '/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_audience_treatment_20241205'

dir_campaign_base_01 = 'dbfs:/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_campaign_base_20241114'
dir_campaign_base_02 = 'dbfs:/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_campaign_base_20241128'
dir_campaign_base_03 = 'dbfs:/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_campaign_base_20241205'

dir_payment_01 = '/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_audience_payment_20241121'
dir_payment_02 = '/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_audience_payment_20241128'
dir_payment_03 = '/mnt/ml-lab/dev_users/dev_sc/audience_list/sms_audience_payment_20241205'

dir_g2 = 'dbfs:/FileStore/mnt/ml-lab/dev_users/dev_sc/bill_reminder_sent_g2_241203.csv'
dir_g3= 'dbfs:/FileStore/mnt/ml-lab/dev_users/dev_sc/bill_reminder_sent_g3_241209.csv'

#### s03 laod data

In [0]:
df_treatment_01 = spark.read.format('delta').load(dir_audience_treatment_01)
df_treatment_02 = spark.read.format('delta').load(dir_audience_treatment_02)
df_treatment_03 = spark.read.format('delta').load(dir_audience_treatment_03)

df_campaign_base_01 = spark.read.format('delta').load(dir_campaign_base_01)
df_campaign_base_02 = spark.read.format('delta').load(dir_campaign_base_02)
df_campaign_base_03 = spark.read.format('delta').load(dir_campaign_base_03)

df_payment_01 = spark.read.format('delta').load(dir_payment_01)
df_payment_02 = spark.read.format('delta').load(dir_payment_02)
df_payment_03 = spark.read.format('delta').load(dir_payment_03)

df_g2_sent = spark.read.format('csv').option('header', 'true').load(dir_g2)
df_g3_sent = spark.read.format('csv').option('header', 'true').load(dir_g3)

In [0]:
display(df_treatment_01.count())

display(df_treatment_02.count())

display(df_treatment_03.count())

In [0]:
display(df_payment_01.limit(10))

In [0]:
display(df_g3_sent.limit(10))

In [0]:
display(
    df_campaign_base_03
    .join(df_treatment_03, on=['fs_acct_id', 'fs_srvc_id', 'L2_combine'], how='inner')
    .join(
        df_g3_sent
        ,  (f.col('fs_cust_id') == f.col('contact_key'))& 
            (f.col('fs_srvc_id') == f.col('service_id')) 
        , 'inner'
    )
    .join(df_payment_03, on=['fs_acct_id', 'fs_srvc_id'], how='inner')
    #.groupBy('L2_combine')
    #.agg(f.countDistinct('fs_acct_id'))
    #.join(df_payment_02, on=['fs_acct_id', 'fs_srvc_id', 'L2_combine'], how='inner')
)

In [0]:
display(
    df_campaign_base_02
    .join(df_treatment_02, on=['fs_acct_id', 'fs_srvc_id', 'L2_combine'], how='inner')
    .join(
        df_g2_sent
        ,  (f.col('fs_cust_id') == f.col('contact_key'))& 
            (f.col('fs_srvc_id') == f.col('service_id')) 
        , 'inner'
    )
    .join(df_payment_02, on=['fs_acct_id', 'fs_srvc_id'], how='inner')
    #.groupBy('L2_combine')
    #.agg(f.countDistinct('fs_acct_id'))
    #.join(df_payment_02, on=['fs_acct_id', 'fs_srvc_id', 'L2_combine'], how='inner')
)